In [507]:
#!L
%pip install seaborn

     |████████████████████████████████| 285 kB 3.4 MB/s 
     |████████████████████████████████| 11.6 MB 6.8 MB/s 
     |████████████████████████████████| 1.1 MB 12.4 MB/s 
     |████████████████████████████████| 14.5 MB 6.1 MB/s 
     |████████████████████████████████| 9.9 MB 14.5 MB/s 
     |████████████████████████████████| 2.2 MB 19.0 MB/s 
     |████████████████████████████████| 67 kB 4.0 MB/s 
     |████████████████████████████████| 227 kB 19.1 MB/s 
     |████████████████████████████████| 510 kB 18.6 MB/s 
     |████████████████████████████████| 25.9 MB 2.1 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mmdet 2.3.0rc0+c6b5ca2 requires Pillow<=6.2.2, but you have pillow 8.0.1 which is incompatible.
You should consider upgrading via the '/usr/bin/python3.7 -m pip install --upgrade pip' command.


In [508]:
#!L
import csv
import json
import numpy as np
import matplotlib.pyplot as plt
import warnings
import pickle
import os
import seaborn as sns

In [509]:
from catboost import CatBoostRegressor, monoforest, Pool
from catboost.utils import create_cd
from sklearn.datasets import load_diabetes
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso, lasso_path
from sklearn.model_selection import train_test_split, GridSearchCV
from tqdm import tqdm_notebook as tqdm
from scipy import stats

In [510]:
RANDOM_STATE = 239

In [511]:
warnings.filterwarnings('ignore')

In [512]:
def save_via_pickle(obj, filepath):
    with open(filepath, 'wb') as output_file:
        pickle.dump(obj, output_file)

In [513]:
def load_from_pickle(filepath):
    with open(filepath, 'rb') as input_file:
        return pickle.load(input_file)

In [514]:
PICKLE_DUMPS_PATH = os.path.join('resources', 'pickle_dumps')

In [515]:
def get_pickle_dump_path(dump_name):
    return os.path.join(PICKLE_DUMPS_PATH, f'{dump_name}.pkl')

In [516]:
def rmse(y_true, y_pred):
    return mean_squared_error(y_true, y_pred, squared=False)

## Loading data

In [ ]:
diabetes = load_diabetes()
diabetes_X, diabetes_y = diabetes['data'], diabetes['target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(diabetes_X, diabetes_y, test_size=0.3, random_state=RANDOM_STATE)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=RANDOM_STATE)

In [ ]:
print(f'Total {X_train.shape[0]} train samples, {X_val.shape[0]} validation_samples, {X_test.shape[0]} test samples')

## Training sklearn GBR

Plan:
* estimate initial set of arguments via grid search on the test dataset

* manually improve the r2 score on the validation dataset

In [ ]:
def train_gradient_boosting_regressor(X_train, y_train, random_state=RANDOM_STATE, arguments_grid=None):
    arguments_grid = arguments_grid if arguments_grid is not None else {
        'n_estimators' : range(100, 1501, 200),
        'max_depth': range(2, 11),
        'learning_rate' : np.hstack([np.arange(0.001, 0.01, 0.002), 
                                     np.arange(0.01, 0.1, 0.02), 
                                     np.arange(0.1, 1., 0.2)])
    }
    reg = GradientBoostingRegressor(random_state=random_state)
    grid_reg = GridSearchCV(reg, arguments_grid, verbose=5, cv=5, n_jobs=4)
    grid_reg.fit(X_train, y_train)
    print(f'Best inital arguments set: {grid_reg.best_params_}')
    return grid_reg.best_params_

In [ ]:
best_initial_params = train_gradient_boosting_regressor(X_train, y_train)

In [517]:
def print_results(reg, X_train, X_val, X_test, y_train, y_val, y_test):
    for X_, y_, label  in [(X_train, y_train, 'Train'), (X_val, y_val, 'Val'), (X_test, y_test, 'Test')]:
        print(f'{label} scores:')
        print(f'R2: {reg.score(X_, y_)}')
        print(f'RMSE: {mean_squared_error(y_, reg.predict(X_), squared=False)}')

In [518]:
def build_plot(x, ys, label_x='', label_y='', title=''):
    plt.clf()
    plt.rcParams["figure.figsize"] = (10, 10)
    plt.xlabel(label_x)
    plt.ylabel(label_y)
    plt.title(title)
    for y_label, y in ys.items():
        plt.plot(x, y, label=y_label)
    plt.legend()
    plt.show()

In [ ]:
def get_scores_for_param_range(model, params, param, param_range, X_train, X_val, y_train, y_val):
    train_scores = []
    test_scores = []
    for param_value in tqdm(param_range):
        current_params = params.copy()
        current_params[param] = param_value
        reg = model(**current_params).fit(X_train, y_train)
        train_scores.append(reg.score(X_train, y_train))
        test_scores.append(reg.score(X_val, y_val))
    return train_scores, test_scores

In [ ]:
def select_best_param_value_from_range(model, current_params, param, param_title, param_range, X_train, X_val,
                                       y_train, y_val):
    train_scores, test_scores = get_scores_for_param_range(model, current_params, param, param_range,
                                                           X_train, X_val, y_train, y_val)
    build_plot(param_range, {'Train r2 score' : train_scores, 'Val r2 score' : test_scores},
               param_title, 'R2 score', f'R2 score for different \'{param_title}\' values')
    best_param_value = param_range[np.argmax(test_scores)]
    if best_param_value != current_params[param]:
        print(f'Changing \'{param_title}\': {current_params[param]} -> {best_param_value}')
        best_params[param] = best_param_value
    return current_params

In [ ]:
inital_reg = GradientBoostingRegressor(**best_initial_params).fit(X_train, y_train)

In [ ]:
print_results(inital_reg, X_train, X_val, X_test, y_train, y_val, y_test)

In [ ]:
best_params = best_initial_params.copy()

In [ ]:
best_params = select_best_param_value_from_range(GradientBoostingRegressor, best_params, 'n_estimators',
                                                 'number of trees', np.arange(100, 2000, 25), X_train, X_val,
                                                 y_train, y_val)

In [ ]:
best_params = select_best_param_value_from_range(GradientBoostingRegressor, best_params, 'learning_rate',
                                                 'learning rate', np.arange(0.02, 0.04, 0.005), X_train, X_val,
                                                 y_train, y_val)

In [ ]:
reg = GradientBoostingRegressor(**best_params).fit(X_train, y_train)

In [ ]:
print_results(reg, X_train, X_val, X_test, y_train, y_val, y_test)

In [ ]:
save_via_pickle((reg, X_train, X_val, X_test, y_train, y_val, y_test),
                get_pickle_dump_path('diabetes_sklearn_gbr'))

## Catboost GBR training

In [ ]:
catboost_reg = CatBoostRegressor()
grid = {
    'learning_rate': [0.001, 0.01, 0.1, 1.],
    'depth':range(1, 9),
}
grid_search_results = catboost_reg.grid_search(grid, X_train, y_train, cv=5, verbose=False, plot=True)

In [ ]:
catboost_reg = CatBoostRegressor(iterations=10000, max_depth=5, l2_leaf_reg=1, 
                                 learning_rate=0.1, use_best_model=True, verbose=False)
                                 # od_type='IncToDec', od_pval=0.05)
catboost_reg.fit(X_train, y_train, eval_set=(X_val, y_val), plot=True)
print_results(catboost_reg, X_train, X_val, X_test, y_train, y_val, y_test)

In [ ]:
save_via_pickle((catboost_reg, X_train, X_val, X_test, y_train, y_val, y_test),
                get_pickle_dump_path('diabetes_catboost_cbr'))

## Extract sklearn monomials

In [ ]:
class Monomial:
    def __init__(self):
        self.features_splits = {}
        self.features = None
        self.thresholds = None

    def add_split(self, split_feature, split_value):
        if split_feature in self.features_splits:
            self.features_splits[split_feature] = max(self.features_splits[split_feature], split_value)
        else:
            self.features_splits[split_feature] = split_value

    def __eq__(self, other):
        if set(self.features_splits.keys()) != set(other.features_splits.keys()):
            return False
        for feature, threshold in self.features_splits.items():
            if not np.isclose(threshold, other.features_splits[feature]):
                return False
        return True

    def finalize(self):
        feature_items = list(self.features_splits.items())
        self.features = np.array([feature_item[0] for feature_item in feature_items], dtype=np.int)
        self.thresholds = np.array([feature_item[1] for feature_item in feature_items])

    def __call__(self, x):
        if self.features is None:
            self.finalize()
        x = np.array(x)
        return 1 if np.all(np.less_equal(self.thresholds, x[self.features])) else 0
    
    def print_to_console(self):
        if len(self.features_splits) == 0:
            print(1, end='')
        for feature, threshold in self.features_splits.items():
            print(f'[x[{feature}] >= {threshold}]', end='')
        print()

In [ ]:
class TreeNode:
    def __init__(self, parent=None, is_left_son=None, feature=None, threshold=None):
        self.parent = parent
        self.is_left_son = is_left_son
        self.feature = feature
        self.threshold = threshold

In [ ]:
def parse_decision_tree(estimator):
    tree = estimator[0].tree_
    n = len(tree.children_left)
    assert n > 0
    nodes = [TreeNode() for _ in range(n)]
    for i in range(n):
        if tree.children_left[i] != tree.children_right[i]:
            nodes[tree.children_left[i]].parent = i
            nodes[tree.children_left[i]].is_left_son = True
            nodes[tree.children_right[i]].parent = i
            nodes[tree.children_right[i]].is_left_son = False
            nodes[i].feature = tree.feature[i]
            nodes[i].threshold = tree.threshold[i]
    return nodes

In [ ]:
def generate_monimals(splits):
    # left sons are optional
    optional_splits = []
    mandatory_splits = []
    for feature, threshold, is_left_son in splits:
        if is_left_son:
            optional_splits.append((feature, threshold))
        else:
            mandatory_splits.append((feature, threshold))
    m = len(optional_splits)
    monomials = []
    for mask in range(2 ** m):
        monomials.append(Monomial())
        for mandatory_split in mandatory_splits:
            monomials[-1].add_split(*mandatory_split)
        for i in range(m):
            if (2 ** i) & mask:
                monomials[-1].add_split(*optional_splits[i])
        monomials[-1].finalize()
    return monomials

In [ ]:
def sift_monomials(monomials):
    # Removes duplicates from monomials (we need it since Monomial isn't hashable)
    result = []
    for monimial in monomials:
        if monimial not in result:
            result.append(monimial)
    return result

In [ ]:
def parse_tree_monomials(tree_estimator):
    nodes = parse_decision_tree(tree_estimator)
    monomials = []
    for node in nodes:
        if node.feature is not None:
            continue
        splits = []
        while node.parent is not None:
            is_left_son = node.is_left_son
            node = nodes[node.parent]
            splits.append((node.feature, node.threshold, is_left_son))
        monomials.extend(generate_monimals(splits))
    return sift_monomials(monomials)

In [ ]:
def parse_ensemble_monomials(tree_ensemble):
    monomials = []
    for estimator in tqdm(tree_ensemble):
        for monomial in parse_tree_monomials(estimator):
            if monomial not in monomials:
                monomials.append(monomial)
    return monomials

In [ ]:
monomials = parse_ensemble_monomials(reg)
print(f'Totaly found {len(monomials)} monomials')

## Extract Catboost monomials

In [ ]:
monoforest.to_polynom(catboost_reg)[0]

In [ ]:
monoforest.to_polynom(catboost_reg)[0].splits[0].__dict__
# dir(monoforest.to_polynom(catboost_reg)[0].splits[0])
# .splits[0]

In [ ]:
split_types = set()

feature_ind = 10000
for monomial in monoforest.to_polynom(catboost_reg):
    for split in monomial.splits:
        split_types.add(split.split_type)
        feature_ind = min(feature_ind, split.feature_idx)

print(feature_ind)
split_types

In [ ]:
def transform_X_catboost(X, monomials):
    X_out = np.ones((X.shape[0], len(monomials)), dtype=np.bool)
    for i, x in enumerate(X):
        for j, monomial in enumerate(monomials):
            for split in monomial.splits:
                X_out[i, j] = X_out[i, j] and np.less_equal(split.border, x[split.feature_idx])
    return X_out

In [ ]:
catboost_monomials = monoforest.to_polynom(catboost_reg)
X_train_catboost = transform_X_catboost(X_train, catboost_monomials)
X_val_catboost = transform_X_catboost(X_val, catboost_monomials)
X_test_catboost = transform_X_catboost(X_test, catboost_monomials)

## Train sklearn lasso

In [ ]:
def transform_X(X, monomials):
    result = np.zeros((X.shape[0], len(monomials)), dtype=np.bool)
    for i, x in enumerate(X):
        for j, m in enumerate(monomials):
            result[i, j] = m(x)
    return result

In [ ]:
X_train_monomial = transform_X(X_train, monomials)
X_val_monomial = transform_X(X_val, monomials)
X_test_monomial = transform_X(X_test, monomials)

In [ ]:
def get_best_lasso_score(X_train, X_test, y_train, y_test, lambda_coef, max_iter=2000):
    best_score = None
    best_iter = None
    for n_iter in tqdm(range(1, max_iter + 1)):
        lasso = Lasso(alpha=lambda_coef, max_iter=n_iter).fit(X_train, y_train)
        new_score = lasso.score(X_test, y_test)
        if best_score is None or best_score < new_score:
            best_score = new_score
            best_iter = n_iter
    return best_score, best_iter

In [ ]:
best_sklearn_score = None
best_sklearn_iter = None
best_sklearn_lambda = None
for lambda_coef in np.arange(0.001, 0.0101, 0.001):
    cur_score, cur_iter = get_best_lasso_score(X_train_monomial, X_test_monomial, y_train, y_test, lambda_coef, max_iter=200)
    print(f'Lambda {lambda_coef}: {cur_iter} iteration(s), R2 score {cur_score}')
    if best_sklearn_score is None or cur_score > best_sklearn_score:
        best_sklearn_score = cur_score
        best_sklearn_iter = cur_iter
        best_sklearn_lambda = lambda_coef

In [ ]:
for lambda_coef in np.arange(0.01, 0.1001, 0.01):
    cur_score, cur_iter = get_best_lasso_score(X_train_monomial, X_test_monomial, y_train, y_test, lambda_coef, max_iter=200)
    print(f'Lambda {lambda_coef}: {cur_iter} iteration(s), R2 score {cur_score}')
    if best_sklearn_score is None or cur_score > best_sklearn_score:
        best_sklearn_score = cur_score
        best_sklearn_iter = cur_iter
        best_sklearn_lambda = lambda_coef

In [ ]:
for lambda_coef in np.arange(0.1, 1.0001, 0.1):
    cur_score, cur_iter = get_best_lasso_score(X_train_monomial, X_test_monomial, y_train, y_test, lambda_coef, max_iter=200)
    print(f'Lambda {lambda_coef}: {cur_iter} iteration(s), R2 score {cur_score}')
    if best_sklearn_score is None or cur_score > best_sklearn_score:
        best_sklearn_score = cur_score
        best_sklearn_iter = cur_iter
        best_sklearn_lambda = lambda_coef

In [ ]:
for lambda_coef in np.arange(0.9, 1.01, 0.01):
    cur_score, cur_iter = get_best_lasso_score(X_train_monomial, X_test_monomial, y_train, y_test, lambda_coef, max_iter=200)
    print(f'Lambda {lambda_coef}: {cur_iter} iteration(s), R2 score {cur_score}')
    if best_sklearn_score is None or cur_score > best_sklearn_score:
        best_sklearn_score = cur_score
        best_sklearn_iter = cur_iter
        best_sklearn_lambda = lambda_coef

## Train catboost lasso (via sklearn)

In [ ]:
best_score = None
best_iter = None
for lambda_coef in np.arange(0.001, 0.0101, 0.001):
    cur_score, cur_iter = get_best_lasso_score(X_train_catboost, X_test_catboost, y_train, y_test, lambda_coef, max_iter=200)
    print(f'Lambda {lambda_coef}: {cur_iter} iteration(s), R2 score {cur_score}')
    if best_score is None or cur_score > best_score:
        best_score = cur_score
        best_iter = cur_iter

In [ ]:
for lambda_coef in np.arange(0.01, 0.101, 0.01):
    cur_score, cur_iter = get_best_lasso_score(X_train_catboost, X_test_catboost, y_train, y_test, lambda_coef, max_iter=200)
    print(f'Lambda {lambda_coef}: {cur_iter} iteration(s), R2 score {cur_score}')
    if best_score is None or cur_score > best_score:
        best_score = cur_score
        best_iter = cur_iter

In [ ]:
for lambda_coef in np.arange(0.1, 1.001, 0.1):
    cur_score, cur_iter = get_best_lasso_score(X_train_catboost, X_test_catboost, y_train, y_test, lambda_coef, max_iter=200)
    print(f'Lambda {lambda_coef}: {cur_iter} iterations, R2 score {cur_score}')
    if best_score is None or cur_score > best_score:
        best_score = cur_score
        best_iter = cur_iter

In [ ]:
for lambda_coef in np.arange(1.1, 2.001, 0.1):
    cur_score, cur_iter = get_best_lasso_score(X_train_catboost, X_test_catboost, y_train, y_test, lambda_coef, max_iter=200)
    print(f'Lambda {lambda_coef}: {cur_iter} iteration(s), R2 score {cur_score}')
    if best_score is None or cur_score > best_score:
        best_score = cur_score
        best_iter = cur_iter

In [ ]:
best_catboost_score = None
best_catboost_lambda = None
best_catboost_iter = None

for lambda_coef in np.arange(1.1, 1.4, 0.03):
    cur_score, cur_iter = get_best_lasso_score(X_train_catboost, X_test_catboost, y_train, y_test, lambda_coef, max_iter=200)
    print(f'Lambda {lambda_coef}: {cur_iter} iteration(s), R2 score {cur_score}')
    if best_catboost_score is None or cur_score > best_catboost_score:
        best_catboost_score = cur_score
        best_catboost_lambda = lambda_coef
        best_catboost_iter = cur_iter

## sklearn confidence intervals

In [ ]:
def build_differences(lasso_params, gradient_boosting_regressor_params, 
                      X_train,  X_train_mon, y_train, X_test, X_test_mon, y_test, n_replicants=1000):
    n = X_test.shape[0]
    diffs = []
    for _ in tqdm(range(n_replicants)):
        ids = np.random.choice(np.arange(n), n, replace=True)
        lasso = Lasso(**lasso_params).fit(X_train_mon[ids], y_train[ids])
        reg = GradientBoostingRegressor(**gradient_boosting_regressor_params).fit(X_train[ids], y_train[ids])
        diffs.append(reg.score(X_test, y_test) - lasso.score(X_test_mon, y_test))
    return diffs

In [ ]:
diffs = build_differences({'alpha': best_sklearn_lambda, 'max_iter': best_sklearn_iter}, best_params,
                          X_train, X_train_monomial, y_train, X_test, X_test_monomial, y_test, n_replicants=500)
np.percentile(diffs, 5), np.percentile(diffs, 95), np.mean(diffs)

In [ ]:
plt.hist(diffs, bins=50)
plt.axvline(np.percentile(diffs, 95), 0, 100, color='r', label='95th percentile')
plt.axvline(np.percentile(diffs, 5), 0, 100, color='g', label='5th percentile')
plt.axvline(np.percentile(diffs, 50), 0, 100, color='orange', label='50th percentile')
plt.legend()
plt.show()

## Catboost confidence intervals

In [ ]:
def build_differences_catboost(lasso_params, 
                               X_train,  X_train_mon, y_train, X_val, y_val,
                               X_test, X_test_mon, y_test, n_replicants=1000):
    n = X_test.shape[0]
    diffs = []
    for _ in tqdm(range(n_replicants)):max_iter=2000
        ids = np.random.choice(np.arange(n), n, replace=True)
        lasso = Lasso(**lasso_params).fit(X_train_mon[ids], y_train[ids])
        catboost_reg = CatBoostRegressor(iterations=10000, max_depth=5, l2_leaf_reg=1, 
                                 learning_rate=0.1, use_best_model=True, verbose=False)
        catboost_reg.fit(X_train[ids], y_train[ids], eval_set=(X_val, y_val))
        diffs.append(reg.score(X_test, y_test) - lasso.score(X_test_mon, y_test))
    return diffs

In [ ]:
catboost_diffs = build_differences_catboost({'alpha': best_catboost_lambda, 'max_iter': best_iter},
                                   X_train, X_train_monomial, y_train, X_val, y_val, X_test, X_test_monomial,
                                   y_test, n_replicants=500)
np.percentile(catboost_diffs, 5), np.percentile(catboost_diffs, 95), np.mean(catboost_diffs)

In [ ]:
plt.hist(catboost_diffs, bins=50)
plt.axvline(np.percentile(catboost_diffs, 95), 0, 100, color='r', label='95th percentile')
plt.axvline(np.percentile(catboost_diffs, 50), 0, 100, color='orange', label='50th percentile')
plt.axvline(np.percentile(catboost_diffs, 5), 0, 100, color='g', label='5th percentile')
plt.legend()
plt.show()

## sklearn scores

### Lasso

In [ ]:
print_results(Lasso(alpha=best_sklearn_lambda, max_iter=best_sklearn_iter).fit(X_train_monomial, y_train),
              X_train_monomial, X_val_monomial, X_test_monomial, y_train, y_val, y_test)

### GBR

In [ ]:
print_results(reg, X_train, X_val, X_test, y_train, y_val, y_test)

## catboost scores

### Lasso

In [ ]:
print_results(Lasso(alpha=best_catboost_lambda, max_iter=best_catboost_iter).fit(X_train_catboost, y_train),
              X_train_catboost, X_val_catboost, X_test_catboost, y_train, y_val, y_test)

### GBR

In [ ]:
print_results(catboost_reg, X_train, X_val, X_test, y_train, y_val, y_test)

## Experiments

In [ ]:
test_diffs = build_differences({'alpha': 2.8, 'max_iter': 12}, best_params,
    X_train, X_train_monomial, y_train, X_test, X_test_monomial, y_test, n_replicants=500)
np.percentile(test_diffs, 5), np.percentile(test_diffs, 95), np.mean(test_diffs)

In [ ]:
with open('../resources/diabetes.pkl', 'rb') as res_file:
    reg_, X_train_, X_val_, X_test_, y_train_, y_val_, y_test_ = pickle.load(res_file)

In [ ]:
monomials_ = parse_ensemble_monomials(reg_)
X_train_mon_ = transform_X(X_train_, monomials_)
X_val_mon_ = transform_X(X_val_, monomials_)
X_test_mon_ = transform_X(X_test_, monomials_)

In [ ]:
best_sklearn_score_ = None
best_sklearn_iter_ = None
best_sklearn_lambda_ = None
for lambda_coef in np.arange(0.001, 0.0101, 0.001):
    cur_score, cur_iter = get_best_lasso_score(X_train_mon_, X_test_mon_, y_train_, y_test_, lambda_coef, max_iter=200)
    print(f'Lambda {lambda_coef}: {cur_iter} iteration(s), R2 score {cur_score}')
    if best_sklearn_score_ is None or cur_score > best_sklearn_score_:
        best_sklearn_score_ = cur_score
        best_sklearn_iter_ = cur_iter
        best_sklearn_lambda_ = lambda_coef

In [ ]:
for lambda_coef in np.arange(0.01, 0.1001, 0.01):
    cur_score, cur_iter = get_best_lasso_score(X_train_mon_, X_test_mon_, y_train_, y_test_, lambda_coef, max_iter=500)
    print(f'Lambda {lambda_coef}: {cur_iter} iteration(s), R2 score {cur_score}')
    if best_sklearn_score_ is None or cur_score > best_sklearn_score_:
        best_sklearn_score_ = cur_score
        best_sklearn_iter_ = cur_iter
        best_sklearn_lambda_ = lambda_coef

In [ ]:
for lambda_coef in np.arange(0.05, 0.0801, 0.003):
    cur_score, cur_iter = get_best_lasso_score(X_train_mon_, X_test_mon_, y_train_, y_test_, lambda_coef, max_iter=500)
    print(f'Lambda {lambda_coef}: {cur_iter} iteration(s), R2 score {cur_score}')
    if best_sklearn_score_ is None or cur_score > best_sklearn_score_:
        best_sklearn_score_ = cur_score
        best_sklearn_iter_ = cur_iter
        best_sklearn_lambda_ = lambda_coef

In [ ]:
for lambda_coef in np.arange(0.1, 1.0001, 0.1):
    cur_score, cur_iter = get_best_lasso_score(X_train_mon_, X_test_mon_, y_train_, y_test_, lambda_coef, max_iter=200)
    print(f'Lambda {lambda_coef}: {cur_iter} iteration(s), R2 score {cur_score}')
    if best_sklearn_score_ is None or cur_score > best_sklearn_score_:
        best_sklearn_score_ = cur_score
        best_sklearn_iter_ = cur_iter
        best_sklearn_lambda_ = lambda_coef

In [ ]:
for lambda_coef in np.arange(1., 10.0001, 1):
    cur_score, cur_iter = get_best_lasso_score(X_train_mon_, X_test_mon_, y_train_, y_test_, lambda_coef, max_iter=200)
    print(f'Lambda {lambda_coef}: {cur_iter} iteration(s), R2 score {cur_score}')
    if best_sklearn_score_ is None or cur_score > best_sklearn_score_:
        best_sklearn_score_ = cur_score
        best_sklearn_iter_ = cur_iter
        best_sklearn_lambda_ = lambda_coef

In [ ]:
for lambda_coef in np.arange(2., 4.0001, 0.2):
    cur_score, cur_iter = get_best_lasso_score(X_train_mon_, X_test_mon_, y_train_, y_test_, lambda_coef, max_iter=200)
    print(f'Lambda {lambda_coef}: {cur_iter} iteration(s), R2 score {cur_score}')
    if best_sklearn_score_ is None or cur_score > best_sklearn_score_:
        best_sklearn_score_ = cur_score
        best_sklearn_iter_ = cur_iter
        best_sklearn_lambda_ = lambda_coef

In [ ]:
n = X_test_.shape[0]
diffs_ = []
for _ in tqdm(range(500)):
    ids = np.random.choice(np.arange(n), n, replace=True)
    lasso = Lasso(alpha=best_sklearn_lambda_, max_iter=best_sklearn_iter_).fit(X_train_mon_[ids], y_train_[ids])
    diffs_.append(reg.score(X_test_[ids], y_test_[ids]) - lasso.score(X_test_mon_[ids], y_test_[ids]))
print(np.percentile(diffs_, 5), np.percentile(diffs_, 95), np.mean(diffs_))

In [ ]:
plt.hist(diffs_, bins=100)
plt.axvline(np.percentile(diffs_, 95), 0, 100, color='r', label='95th percentile')
plt.axvline(np.percentile(diffs_, 5), 0, 100, color='g', label='5th percentile')
plt.axvline(np.percentile(diffs_, 50), 0, 100, color='orange', label='50th percentile')
plt.legend()
plt.show()

In [ ]:
print_results(reg_, X_train_, X_val_, X_test_, y_train_, y_val_, y_test_)

In [ ]:
print_results(Lasso(alpha=best_sklearn_lambda_, max_iter=best_sklearn_iter_).fit(X_train_mon_, y_train_), 
              X_train_mon_, X_val_mon_, X_test_mon_, y_train_, y_val_, y_test_)

In [ ]:
best_sklearn_score_val = None
best_sklearn_iter_val = None
best_sklearn_lambda_val = None
for lambda_coef in np.arange(0.001, 0.0101, 0.001):
    cur_score, cur_iter = get_best_lasso_score(X_train_monomial, X_val_monomial, y_train, y_val, lambda_coef, max_iter=200)
    print(f'Lambda {lambda_coef}: {cur_iter} iteration(s), R2 score {cur_score}')
    if best_sklearn_score_val is None or cur_score > best_sklearn_score_val:
        best_sklearn_score_val = cur_score
        best_sklearn_iter_val = cur_iter
        best_sklearn_lambda_val = lambda_coef

In [ ]:
for lambda_coef in np.arange(0.01, 0.1001, 0.01):
    cur_score, cur_iter = get_best_lasso_score(X_train_monomial, X_val_monomial, y_train, y_val, lambda_coef, max_iter=200)
    print(f'Lambda {lambda_coef}: {cur_iter} iteration(s), R2 score {cur_score}')
    if best_sklearn_score_val is None or cur_score > best_sklearn_score_val:
        best_sklearn_score_val = cur_score
        best_sklearn_iter_val = cur_iter
        best_sklearn_lambda_val = lambda_coef

In [ ]:
for lambda_coef in np.arange(0.1, 1.0001, 0.1):
    cur_score, cur_iter = get_best_lasso_score(X_train_monomial, X_val_monomial, y_train, y_val, lambda_coef, max_iter=200)
    print(f'Lambda {lambda_coef}: {cur_iter} iteration(s), R2 score {cur_score}')
    if best_sklearn_score_val is None or cur_score > best_sklearn_score_val:
        best_sklearn_score_val = cur_score
        best_sklearn_iter_val = cur_iter
        best_sklearn_lambda_val = lambda_coef

In [ ]:
for lambda_coef in np.arange(0.8, 2.01, 0.1):
    cur_score, cur_iter = get_best_lasso_score(X_train_monomial, X_val_monomial, y_train, y_val, lambda_coef, max_iter=200)
    print(f'Lambda {lambda_coef}: {cur_iter} iteration(s), R2 score {cur_score}')
    if best_sklearn_score_val is None or cur_score > best_sklearn_score_val:
        best_sklearn_score_val = cur_score
        best_sklearn_iter_val = cur_iter
        best_sklearn_lambda_val = lambda_coef

In [ ]:
for lambda_coef in np.arange(1.2, 1.4001, 0.02):
    cur_score, cur_iter = get_best_lasso_score(X_train_monomial, X_val_monomial, y_train, y_val, lambda_coef, max_iter=200)
    print(f'Lambda {lambda_coef}: {cur_iter} iteration(s), R2 score {cur_score}')
    if best_sklearn_score_val is None or cur_score > best_sklearn_score_val:
        best_sklearn_score_val = cur_score
        best_sklearn_iter_val = cur_iter
        best_sklearn_lambda_val = lambda_coef

In [ ]:
diffs_val = build_differences({'alpha': best_sklearn_lambda_val, 'max_iter': best_sklearn_iter_val}, best_params,
                          X_train, X_train_monomial, y_train, X_test, X_test_monomial, y_test, n_replicants=500)
np.percentile(diffs_val, 5), np.percentile(diffs_val, 95), np.mean(diffs_val)

In [ ]:
plt.hist(diffs_val, bins=50)
plt.axvline(np.percentile(diffs_val, 95), 0, 100, color='r', label='95th percentile')
plt.axvline(np.percentile(diffs_val, 5), 0, 100, color='g', label='5th percentile')
plt.axvline(np.percentile(diffs_val, 50), 0, 100, color='orange', label='50th percentile')
plt.legend()
plt.show()

In [ ]:
def save_dataset_tsv(X, y, output_file, feature_names=None, write_header=True):
    feature_names = feature_names if feature_names is not None else [f'F{i}' for i in range(X.shape[1])]
    feature_names.append('target')
    with open(output_file, 'w') as tsv_file:
        writer = csv.DictWriter(tsv_file, fieldnames = feature_names, delimiter='\t')
        if write_header:
            writer.writeheader()
        for x_ind, x in enumerate(X):
            row = {feature_names[i] : x[i] for i in range(x.shape[0])}
            row['target'] = y[x_ind]
            writer.writerow(row)

In [ ]:
def save_dataset_for_catboost_lasso(X, y, model, output_dir, json_filepath):
    os.makedirs(output_dir, exist_ok=True)
    save_dataset_tsv(X, y, os.path.join(output_dir, 'dataset.csv'), write_header=False)
    model.save_model(os.path.join(output_dir, 'model.bin'))
    create_cd(label=X.shape[1], output_path=os.path.join(output_dir, 'dataset.cd'))
    with open(json_filepath, 'w') as json_file:
        json.dump({'ds_path': os.path.abspath(os.path.join(output_dir, 'dataset.csv')),
                   'model_path': os.path.abspath(os.path.join(output_dir, 'model.bin')),
                   'cd_path': os.path.abspath(os.path.join(output_dir, 'dataset.cd'))}, json_file)

In [ ]:
catboost_reg, X_train, X_val, X_test, y_train, y_val, y_test = load_from_pickle(get_pickle_dump_path('diabetes_catboost_cbr'))

In [ ]:
save_dataset_for_catboost_lasso(X_train, y_train, catboost_reg, 
                          os.path.join('..', 'datasets', 'catboost', 'diabetes', 'train'),
                          os.path.join('..', 'datasets', 'catboost', 'diabetes', 'train.json'))

In [ ]:
save_dataset_for_catboost_lasso(X_test, y_test, catboost_reg, 
                          os.path.join('..', 'datasets', 'catboost', 'diabetes', 'test'),
                          os.path.join('..', 'datasets', 'catboost', 'diabetes', 'test.json'))

In [ ]:
save_dataset_for_catboost_lasso(X_val, y_val, catboost_reg, 
                          os.path.join('..', 'datasets', 'catboost', 'diabetes', 'val'),
                          os.path.join('..', 'datasets', 'catboost', 'diabetes', 'val.json'))

In [ ]:
def save_bootstrap_dataset_for_catboost_lasso(X, y, model, output_dir, n_samples=500):
    n = X.shape[0]
    for i in tqdm(range(n_samples)):
        ids = np.random.choice(np.arange(n), n, replace=True)
        sample_dir = os.path.join(output_dir, f'sample_{i}')
        os.makedirs(sample_dir, exist_ok=True)
        save_dataset_for_catboost_lasso(X[ids], y[ids], model, sample_dir,
                                        os.path.join(output_dir, f'sample_{i}.json'))

In [ ]:
save_bootstrap_dataset_for_catboost_lasso(X_train, y_train, catboost_reg,
                                          os.path.join('..', 'datasets', 'catboost', 'diabetes', 'bootstrap'))

In [ ]:
def train_lasso(n_iterations, lambda_arg, X, y):
    return Lasso(max_iter=n_iterations, alpha=lambda_arg).fit(X, y)

def select_lasso_iter(X_train, X_val, y_train, y_val, lambda_arg, max_iter=2000, iter_step=50):
    best_score = None
    best_iter = None
    
    for cur_iter in range(1, max_iter + 1, iter_step):
        lasso = train_lasso(cur_iter, lambda_arg, X_train, y_train)
        cur_score = mean_squared_error(y_val, lasso.predict(X_val), squared=False)
        if best_score is None or cur_score < best_score:
            best_score = cur_score
            best_iter = cur_iter
    
    return best_score, best_iter

def select_lasso_lambda(X_train, X_val, y_train, y_val, lambda_max=1000, lambda_min=1e-4, decay=0.9,
                        max_iter=2000, iter_step=50):
    cur_lambda = lambda_max
    best_score = None
    best_lambda = None
    best_iter = None
    iter_number = 0
    while np.less(lambda_min, cur_lambda):
        cur_score, cur_iter = select_lasso_iter(X_train, X_val, y_train, y_val, cur_lambda,
                                                max_iter, iter_step)
        if best_score is None or cur_score < best_score:
            best_score = cur_score
            best_iter = cur_iter
            best_lambda = cur_lambda
            print(f'New best score: lambda={cur_lambda}, n_iter={cur_iter}, rmse={cur_score}')
        cur_lambda *= decay
        iter_number += 1
        if iter_number % 10 == 0:
            print(f'Iter {iter_number}(lambda={cur_lambda}), best: lambda={best_lambda}, n_iter={best_iter}, rmse={best_score}')
    
    return best_lambda, best_iter, best_score

In [ ]:
def bootstrap_scores(catboost_params, lasso_params, 
                     X_train, X_train_mon, 
                     X_val, X_val_mon,
                     X_test, X_test_mon,
                     y_train, y_val, y_test, 
                     n_samples=500):
    n = X_train.shape[0]
    
    all_ids = []
    diffs = []
    val_diffs = []
    for _ in tqdm(range(n_samples)):
        ids = np.random.choice(np.arange(n), n, replace=True)
        lasso = Lasso(**lasso_params).fit(X_train_mon[ids], y_train[ids])
        reg = CatBoostRegressor(**catboost_params)
        reg.fit(X_train[ids], y_train[ids], eval_set=(X_val, y_val), verbose=False, plot=False)
        diffs.append(rmse(y_test, reg.predict(X_test)) - rmse(y_test, lasso.predict(X_test_mon)))
        val_diffs.append(rmse(y_val, reg.predict(X_val)) - rmse(y_val, lasso.predict(X_val_mon)))
        all_ids.append(ids)
    
    sns.distplot(diffs, hist=True, kde=True, 
                 bins=25, color = 'darkblue', 
                 hist_kws={'edgecolor':'black'},
                 kde_kws={'linewidth': 4},
                 axlabel='rmse')
    return diffs, val_diffs, all_ids

## Catboost experiments

In [ ]:
print_results(catboost_reg, X_train, X_val, X_test, y_train, y_val, y_test)

In [ ]:
catboost_argument_grid = {
    'learning_rate': np.hstack([np.arange(0.001, 0.01, 0.001), 
                                np.arange(0.01, 0.1, 0.01), 
                                np.arange(0.1, 0.2, 0.01)]),
    'depth':range(1, 9),
}

## Base tree

In [ ]:
base_catboost_reg = CatBoostRegressor(boosting_type='Plain', bootstrap_type='No',
                                      score_function='L2', leaf_estimation_method=None)

In [ ]:
grid_search_results = base_catboost_reg.grid_search(catboost_argument_grid, X_train, y_train, 
                                                    cv=5, verbose=False, plot=True)

In [ ]:
base_catboost_reg.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=False, plot=True)
print_results(base_catboost_reg, X_train, X_val, X_test, y_train, y_val, y_test)

In [ ]:
regs = {}

In [ ]:
split_types = set()

feature_ind = 10000
for monomial in monoforest.to_polynom(catboost_reg):
    for split in monomial.splits:
        split_types.add(split.split_type)

print(split_types)

regs['base'] = {'reg': base_catboost_reg}
regs['base']['monomials'] = monoforest.to_polynom(base_catboost_reg)
regs['base']['X_train'] = transform_X_catboost(X_train, regs['base']['monomials'])
regs['base']['X_val'] = transform_X_catboost(X_val, regs['base']['monomials'])
regs['base']['X_test'] = transform_X_catboost(X_test, regs['base']['monomials'])
regs['base']['params'] = {
    'boosting_type': 'Plain',
    'bootstrap_type': 'No',
    'score_function': 'L2',
    'leaf_estimation_method': None,
    'depth': 5, 
    'learning_rate': 0.15
}

In [ ]:
select_lasso_lambda(regs['base']['X_train'], regs['base']['X_val'], y_train, y_val, decay=0.9,
                    iter_step=10)

In [ ]:
select_lasso_lambda(regs['base']['X_train'], regs['base']['X_val'], y_train, y_val, decay=0.9,
                    iter_step=10, lambda_max=2, lambda_min=1e-2, max_iter=200)

In [ ]:
select_lasso_iter(regs['base']['X_train'], regs['base']['X_val'], 
                  y_train, y_val, 1.4580000000000002, max_iter=200, iter_step=1)

In [ ]:
lasso_params = {'alpha' : 1.4580000000000002,
                'max_iter': 7}

In [ ]:
diffs, val_diffs, ids = bootstrap_scores(regs['base']['params'], lasso_params, 
                                         X_train, regs['base']['X_train'], 
                                         X_val, regs['base']['X_val'],
                                         X_test, regs['base']['X_test'],
                                         y_train, y_val, y_test, n_samples=500)

In [ ]:
np.percentile(diffs, 5), np.percentile(diffs, 90), np.percentile(diffs, 50) 

## Default tree

In [ ]:
default_catboost_reg = CatBoostRegressor()
grid_search_results = default_catboost_reg.grid_search(catboost_argument_grid, X_train, y_train, 
                                                       cv=5, verbose=False, plot=True)

In [ ]:
regs['default'] = {
    'depth': 3, 
    'learning_rate': 0.13
}

In [ ]:
default_catboost_reg.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=False, plot=True)
print_results(base_catboost_reg, X_train, X_val, X_test, y_train, y_val, y_test)